In [2]:
import pandas as pd
import csv
import json
import pymongo
from pymongo import MongoClient
import warnings
warnings.filterwarnings('ignore')

In [4]:
fort1000_file = "Data/Fortune1000-final.csv"
fort1000_df = pd.read_csv(fort1000_file, encoding = 'ISO-8859-1')
fort1000_df.head()

,rank,title,Previous Rank,Revenues ($M),Revenue Change,Profits ($M),Profit Change,Assets ($M),Mkt Value as of 3/29/18 ($M),Employees,CEO,CEO Title,Sector,Industry,Years on Fortune 500 List,City,State,Latitude,Longitude
0,1,Walmart,1,"$500,343",3.00%,"$9,862.00",-27.70%,"$204,522","$263,563","2,300,000",C. Douglas McMillon,"President, Chief Executive Officer & Director",Retailing,General Merchandisers,24,Bentonville,AR,36.372854,-94.208817
1,2,Exxon Mobil,4,"$244,363",17.40%,"$19,710.00",151.40%,"$348,691","$316,157","71,200",Darren W. Woods,Chairman & Chief Executive Officer,Energy,Petroleum Refining,24,Irving,TX,32.814018,-96.948894
2,3,Berkshire Hathaway,2,"$242,137",8.30%,"$44,940.00",86.70%,"$702,095","$492,008","377,000",Warren E. Buffett,"Chairman, President & Chief Executive Officer",Financials,Insurance: Property and Casualty (Stock),24,Omaha,NE,41.256537,-95.934503
3,4,Apple,3,"$229,234",6.30%,"$48,351.00",5.80%,"$375,319","$851,318","123,000",Timothy D. Cook,Chairman & Chief Executive Officer,Technology,"Computers, Office Equipment",24,Cupertino,CA,37.322998,-122.032182
4,5,UnitedHealth Group,6,"$201,159",8.80%,"$10,558.00",50.50%,"$139,058","$207,080","260,000",David S. Wichmann,Chairman & Chief Executive Officer,Health Care,Health Care: Insurance and Managed Care,24,Minnetonka,MN,44.921184,-93.468749


In [5]:
fort1000_reduced = fort1000_df[['title', 'Sector', 'Industry', 'City', 'State', 'Latitude', 'Longitude']]
fort1000_reduced.rename(columns = {'title': 'Company Name'}, inplace=True)

fort1000_reduced.head()

,Company Name,Sector,Industry,City,State,Latitude,Longitude
0,Walmart,Retailing,General Merchandisers,Bentonville,AR,36.372854,-94.208817
1,Exxon Mobil,Energy,Petroleum Refining,Irving,TX,32.814018,-96.948894
2,Berkshire Hathaway,Financials,Insurance: Property and Casualty (Stock),Omaha,NE,41.256537,-95.934503
3,Apple,Technology,"Computers, Office Equipment",Cupertino,CA,37.322998,-122.032182
4,UnitedHealth Group,Health Care,Health Care: Insurance and Managed Care,Minnetonka,MN,44.921184,-93.468749


In [6]:
fort500_file = "Data/fortune500.csv"
fort500_df = pd.read_csv(fort500_file, encoding = 'ISO-8859-1')
fort500_df.head()

,Rank,Company Name,Number of Employees,Change in Rank,Revenues ($millions),Revenue Change,Profits ($millions),Profit Change,Assets ($millions),Market Value As of 3/31/20 ($m)
0,1,Walmart,"2,200,000",-,"$523,964",1.90%,"$14,881.00",123.10%,"$236,495","$321,803"
1,2,Amazon.com,"798,000",3,"$280,522",20.50%,"$11,588.00",15.00%,"$225,248","$970,680"
2,3,Exxon Mobil,"74,900",-1,"$264,938",-8.70%,"$14,340.00",-31.20%,"$362,597","$160,696"
3,4,Apple,"137,000",-1,"$260,174",-2.00%,"$55,256.00",-7.20%,"$338,516","$1,112,641"
4,5,CVS Health,"290,000",3,"$256,776",32.00%,"$6,634.00",-,"$222,449","$77,376"


In [7]:
fort500_final = pd.merge(fort500_df, fort1000_reduced, how="left", on=['Company Name', 'Company Name'])
fort500_final

,Rank,Company Name,Number of Employees,Change in Rank,Revenues ($millions),Revenue Change,Profits ($millions),Profit Change,Assets ($millions),Market Value As of 3/31/20 ($m),Sector,Industry,City,State,Latitude,Longitude
0,1,Walmart,"2,200,000",-,"$523,964",1.90%,"$14,881.00",123.10%,"$236,495","$321,803",Retailing,General Merchandisers,Bentonville,AR,36.372854,-94.208817
1,2,Amazon.com,"798,000",3,"$280,522",20.50%,"$11,588.00",15.00%,"$225,248","$970,680",Retailing,Internet Services and Retailing,Seattle,WA,47.606209,-122.332071
2,3,Exxon Mobil,"74,900",-1,"$264,938",-8.70%,"$14,340.00",-31.20%,"$362,597","$160,696",Energy,Petroleum Refining,Irving,TX,32.814018,-96.948894
3,4,Apple,"137,000",-1,"$260,174",-2.00%,"$55,256.00",-7.20%,"$338,516","$1,112,641",Technology,"Computers, Office Equipment",Cupertino,CA,37.322998,-122.032182
4,5,CVS Health,"290,000",3,"$256,776",32.00%,"$6,634.00",-,"$222,449","$77,376",Health Care,Health Care: Pharmacy and Other Services,Woonsocket,RI,42.002876,-71.514784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,Crown Castle International,"5,100",-,"$5,763",7.30%,$860.00,38.30%,"$38,457","$60,179",Financials,Real estate,Houston,TX,29.760427,-95.369803
496,497,Simon Property Group,"3,750",-1,"$5,755",1.70%,"$2,101.60",-13.90%,"$31,232","$16,835",Financials,Real estate,Indianapolis,IN,39.768403,-86.158068
497,498,Cerner,"27,400",-,"$5,693",6.10%,$529.50,-16.00%,"$6,895","$19,164",Health Care,Health Care: Pharmacy and Other Services,North Kansas City,MO,39.142908,-94.572978
498,499,Post Holdings,"10,100",-28,"$5,681",-9.20%,$124.70,-73.30%,"$11,952","$5,732","Food, Beverages & Tobacco",Food Consumer Products,St. Louis,MO,38.627003,-90.199404


In [8]:
fort500_final.to_csv("Data/Fortune500_final.csv")

In [9]:
csvfile = open('Data/fortune500_final.csv', encoding = 'ISO-8859-1')
reader = csv.DictReader( csvfile )
mongo_client=MongoClient('mongodb://localhost:27017/fortune500') 
db=mongo_client.fortune500
db.companyList.drop()
header= [ "Rank", "Company Name", "Number of Employees", "Change in Rank",
            "Revenues ($millions)", "Revenue Change", "Profits ($millions)",
            "Profit Change", "Assets ($millions)", "Market Value As of 3/31/20 ($m)",
            "Sector", "Industry", "City", "State" ,"Latitude", "Longitude"]

for each in reader:
    row={}
    for field in header:
        row[field]=each[field]

    db.companyList.insert(row)